In [1]:
import numpy as np
import torch
import torch.nn as nn
import zipfile
import pandas as pd
import glob
import albumentations as A
from albumentations.pytorch import ToTensorV2

In [2]:
pip install albumentations

In [3]:
with zipfile.ZipFile("/Users/Yashwanth/isic-2024-challenge.zip") as z:
    with z.open("train-metadata.csv") as f:
        train_df = pd.read_csv(f)
    with z.open("test-metadata.csv") as f:
        test_df = pd.read_csv(f)

all_df = pd.concat([train_df, test_df]).reset_index(drop=True) 
display(train_df.head())
display(test_df.head())

C:\Users\Yashwanth\AppData\Local\Temp\ipykernel_7068\2079818307.py:3: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(f)


,isic_id,target,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,...,lesion_id,iddx_full,iddx_1,iddx_2,iddx_3,iddx_4,iddx_5,mel_mitotic_index,mel_thick_mm,tbp_lv_dnn_lesion_confidence
0,ISIC_0015670,0,IP_1235828,60.0,male,lower extremity,3.04,TBP tile: close-up,3D: white,20.244422,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,97.517282
1,ISIC_0015845,0,IP_8170065,60.0,male,head/neck,1.10,TBP tile: close-up,3D: white,31.712570,...,IL_6727506,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,3.141455
2,ISIC_0015864,0,IP_6724798,60.0,male,posterior torso,3.40,TBP tile: close-up,3D: XP,22.575830,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.804040
3,ISIC_0015902,0,IP_4111386,65.0,male,anterior torso,3.22,TBP tile: close-up,3D: XP,14.242329,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.989998
4,ISIC_0024200,0,IP_8313778,55.0,male,anterior torso,2.73,TBP tile: close-up,3D: white,24.725520,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,70.442510


,isic_id,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,tbp_lv_Aext,...,tbp_lv_radial_color_std_max,tbp_lv_stdL,tbp_lv_stdLExt,tbp_lv_symm_2axis,tbp_lv_symm_2axis_angle,tbp_lv_x,tbp_lv_y,tbp_lv_z,attribution,copyright_license
0,ISIC_0015657,IP_6074337,45.0,male,posterior torso,2.70,TBP tile: close-up,3D: XP,22.80433,20.007270,...,0.304827,1.281532,2.299935,0.479339,20,-155.06510,1511.222000,113.980100,Memorial Sloan Kettering Cancer Center,CC-BY
1,ISIC_0015729,IP_1664139,35.0,female,lower extremity,2.52,TBP tile: close-up,3D: XP,16.64867,9.657964,...,0.000000,1.271940,2.011223,0.426230,25,-112.36924,629.535889,-15.019287,"Frazer Institute, The University of Queensland...",CC-BY
2,ISIC_0015740,IP_7142616,65.0,male,posterior torso,3.16,TBP tile: close-up,3D: XP,24.25384,19.937380,...,0.230742,1.080308,2.705857,0.366071,110,-84.29282,1303.978000,-28.576050,FNQH Cairns,CC-BY


In [4]:
df = train_df.copy()
print(f"Number of images: {df.shape[0]}, number of positive cases: {df.target.sum()}, number of negative cases: {(df['target'] == 0).sum()}, number of patients: {df['patient_id'].nunique()}")
df_positive = df[df["target"] == 1].reset_index(drop=True)
df_negative = df[df["target"] == 0].reset_index(drop=True)

Number of images: 401059, number of positive cases: 393, number of negative cases: 400666, number of patients: 1042


Note that number of images is less than number of patients, which means one patient could have multiple pictures.

ADA

In [5]:
print(df.columns)

Index(['isic_id', 'target', 'patient_id', 'age_approx', 'sex',
       'anatom_site_general', 'clin_size_long_diam_mm', 'image_type',
       'tbp_tile_type', 'tbp_lv_A', 'tbp_lv_Aext', 'tbp_lv_B', 'tbp_lv_Bext',
       'tbp_lv_C', 'tbp_lv_Cext', 'tbp_lv_H', 'tbp_lv_Hext', 'tbp_lv_L',
       'tbp_lv_Lext', 'tbp_lv_areaMM2', 'tbp_lv_area_perim_ratio',
       'tbp_lv_color_std_mean', 'tbp_lv_deltaA', 'tbp_lv_deltaB',
       'tbp_lv_deltaL', 'tbp_lv_deltaLB', 'tbp_lv_deltaLBnorm',
       'tbp_lv_eccentricity', 'tbp_lv_location', 'tbp_lv_location_simple',
       'tbp_lv_minorAxisMM', 'tbp_lv_nevi_confidence', 'tbp_lv_norm_border',
       'tbp_lv_norm_color', 'tbp_lv_perimeterMM',
       'tbp_lv_radial_color_std_max', 'tbp_lv_stdL', 'tbp_lv_stdLExt',
       'tbp_lv_symm_2axis', 'tbp_lv_symm_2axis_angle', 'tbp_lv_x', 'tbp_lv_y',
       'tbp_lv_z', 'attribution', 'copyright_license', 'lesion_id',
       'iddx_full', 'iddx_1', 'iddx_2', 'iddx_3', 'iddx_4', 'iddx_5',
       'mel_mitotic_index', '

To see five patients' information

In [6]:
 df.T.sample(n=5, axis=1)

,307520,139670,290810,92699,63337
isic_id,ISIC_7689287,ISIC_3536990,ISIC_7277339,ISIC_2369418,ISIC_1637187
target,0,0,0,0,0
patient_id,IP_9778433,IP_5580991,IP_5580991,IP_7797815,IP_1872392
age_approx,60.0,60.0,60.0,75.0,60.0
sex,male,male,male,male,female
anatom_site_general,posterior torso,lower extremity,posterior torso,anterior torso,anterior torso
clin_size_long_diam_mm,2.59,3.54,2.65,3.18,3.2
image_type,TBP tile: close-up,TBP tile: close-up,TBP tile: close-up,TBP tile: close-up,TBP tile: close-up
tbp_tile_type,3D: XP,3D: XP,3D: XP,3D: white,3D: XP
tbp_lv_A,20.86375,11.12719,25.09333,24.59704,21.069363


Checking NaN values

In [7]:
df.isna().sum()

isic_id                              0
target                               0
patient_id                           0
age_approx                        2798
sex                              11517
anatom_site_general               5756
clin_size_long_diam_mm               0
image_type                           0
tbp_tile_type                        0
tbp_lv_A                             0
tbp_lv_Aext                          0
tbp_lv_B                             0
tbp_lv_Bext                          0
tbp_lv_C                             0
tbp_lv_Cext                          0
tbp_lv_H                             0
tbp_lv_Hext                          0
tbp_lv_L                             0
tbp_lv_Lext                          0
tbp_lv_areaMM2                       0
tbp_lv_area_perim_ratio              0
tbp_lv_color_std_mean                0
tbp_lv_deltaA                        0
tbp_lv_deltaB                        0
tbp_lv_deltaL                        0
tbp_lv_deltaLB           

we can see that majority of idd_2, iddx_3,iddx_4,iddx_4, iddx_5, mel_motiotic_index,mel_thick_mm are none, therefore we can drop these columns
as well as some other columns 


In [11]:
def drop_columns(df):
    columns = ['isic_id', 'copyright_license', 'attribution', 'image_type', 'iddx_1', 'iddx_2', 'iddx_3', 'iddx_4',
               'iddx_5', 'iddx_full', 'mel_mitotic_index', 'mel_thick_mm', 'tbp_tile_type', 'tbp_lv_dnn_lesion_confidence',
               'lesion_id']

    for column in columns:
        try:
            df.drop(column, axis=1, inplace=True)
            print(f"Dropped column: {column}")
        except KeyError as e:
            print('No column with name', e)

    return df


In [12]:
df_new=drop_columns(df)

No column with name "['isic_id'] not found in axis"
No column with name "['copyright_license'] not found in axis"
No column with name "['attribution'] not found in axis"
No column with name "['image_type'] not found in axis"
No column with name "['iddx_1'] not found in axis"
No column with name "['iddx_2'] not found in axis"
No column with name "['iddx_3'] not found in axis"
No column with name "['iddx_4'] not found in axis"
No column with name "['iddx_5'] not found in axis"
No column with name "['iddx_full'] not found in axis"
No column with name "['mel_mitotic_index'] not found in axis"
No column with name "['mel_thick_mm'] not found in axis"
No column with name "['tbp_tile_type'] not found in axis"
No column with name "['tbp_lv_dnn_lesion_confidence'] not found in axis"
No column with name "['lesion_id'] not found in axis"


Filling Na with modes 

In [13]:
df_new.isna().sum()
#we can repalce NA values with mode
modes=df.mode().iloc[0]
df_new.fillna(modes, inplace=True)

Next, we would like to seperate all the variables as continuous variables and categorical variables

In [14]:
cat_df=['patient_id','target','age_approx','sex','anatom_site_general','tbp_lv_location','tbp_lv_location_simple']
con_df=[x for x in df_new.columns if x not in cat_df]
for col in cat_df:
  df_new[col]=df_new[col].astype('category').cat.codes


Next we proceed with the random forest 

In [15]:
from numpy import random
from sklearn.model_selection import train_test_split
X = df_new.drop(columns=['target'])
y = df_new['target']
X_train, X_test,y_train, y_test = train_test_split (X,y,test_size=0.25,shuffle=True, stratify=df_new['target'])